<a href="https://colab.research.google.com/github/roscoekerby/opencv/blob/main/Image_Filtering_(Edge_Detection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import sys
import numpy

PREVIEW  = 0  # Preview Mode
BLUR     = 1  # Blurring Filter
FEATURES = 2  # Corner Feature Detector
CANNY    = 3  # Canny Edge Detector

feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)
s = 0
if len(sys.argv) > 1:
    s = sys.argv[1]

image_filter = PREVIEW
alive = True

win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

source = cv2.VideoCapture(s)

while alive:
    has_frame, frame = source.read()
    if not has_frame:
        break

    frame = cv2.flip(frame, 1)

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == CANNY:
        result = cv2.Canny(frame, 80, 150)
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13))
    elif image_filter == FEATURES:
        result = frame
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
        if corners is not None:
            for x, y in numpy.float32(corners).reshape(-1, 2):
                cv2.circle(result, (x, y), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False
    elif key == ord("C") or key == ord("c"):
        image_filter = CANNY
    elif key == ord("B") or key == ord("b"):
        image_filter = BLUR
    elif key == ord("F") or key == ord("f"):
        image_filter = FEATURES
    elif key == ord("P") or key == ord("p"):
        image_filter = PREVIEW

source.release()
cv2.destroyWindow(win_name)

Fixed Features

In [ ]:
import cv2
import sys
import numpy as np

PREVIEW  = 0  # Preview Mode
BLUR     = 1  # Blurring Filter
FEATURES = 2  # Corner Feature Detector
CANNY    = 3  # Canny Edge Detector

feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)
s = 0
if len(sys.argv) > 1:
    s = sys.argv[1]

image_filter = PREVIEW
alive = True

win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

source = cv2.VideoCapture(s)

while alive:
    has_frame, frame = source.read()
    if not has_frame:
        break

    frame = cv2.flip(frame, 1)

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == CANNY:
        result = cv2.Canny(frame, 80, 150)
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13))
    elif image_filter == FEATURES:
        result = frame
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
        if corners is not None:
            for x, y in np.float32(corners).reshape(-1, 2):
                cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False
    elif key == ord("C") or key == ord("c"):
        image_filter = CANNY
    elif key == ord("B") or key == ord("b"):
        image_filter = BLUR
    elif key == ord("F") or key == ord("f"):
        image_filter = FEATURES
    elif key == ord("P") or key == ord("p"):
        image_filter = PREVIEW

source.release()
cv2.destroyWindow(win_name)


In [ ]:
import cv2
import numpy as np
from base64 import b64decode
from IPython.display import display, Javascript
from google.colab.output import eval_js, register_callback

# JavaScript for video recording
def record_video(filename='video.webm'):
    js = Javascript('''
    async function recordVideo() {
        const div = document.createElement('div');

        // Create video element to display the video feed
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        // Create buttons for start and stop recording
        const recordButton = document.createElement('button');
        recordButton.textContent = 'Start Recording';
        div.appendChild(recordButton);

        const stopButton = document.createElement('button');
        stopButton.textContent = 'Stop Recording';
        stopButton.style.display = 'none'; // Hide stop button initially
        div.appendChild(stopButton);

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        let recorder = new MediaRecorder(stream, {
            mimeType: 'video/webm; codecs="vp8"'
        });
        let recordingData = [];

        recordButton.onclick = function() {
            recorder.start();
            recordButton.textContent = 'Recording...';
            recordButton.disabled = true;
            stopButton.style.display = 'inline-block';
        }

        stopButton.onclick = function() {
            recorder.stop();
            recordButton.textContent = 'Start Recording';
            recordButton.disabled = false;
            stopButton.style.display = 'none';
        }

        recorder.ondataavailable = e => {
            recordingData.push(e.data);
        };

        recorder.onstop = async () => {
            const blob = new Blob(recordingData, { type: 'video/webm' });
            stream.getVideoTracks()[0].stop();
            div.remove();

            const reader = new FileReader();
            reader.onload = () => {
                google.colab.kernel.invokeFunction('notebook.save_video', [reader.result], {});
            };
            reader.readAsDataURL(blob);
        };
    }
    recordVideo();
    ''')

    display(js)

# Python function to handle saving the video file
def save_video(data, filename='video.webm'):
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    print(f'Video saved as {filename}')

# Register the Python function as a callback
register_callback('notebook.save_video', save_video)

# Execute recording and saving process
record_video()

# Define the filters
def apply_filters(input_path, output_path):
    PREVIEW  = 0  # Preview Mode
    BLUR     = 1  # Blurring Filter
    FEATURES = 2  # Corner Feature Detector
    CANNY    = 3  # Canny Edge Detector

    feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)
    image_filter = CANNY  # Change this to BLUR, FEATURES, or PREVIEW to apply different filters

    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'vp80')  # Use 'XVID' or another codec if preferred
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (640, 480))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)

        if image_filter == PREVIEW:
            result = frame
        elif image_filter == CANNY:
            result = cv2.Canny(frame, 80, 150)
        elif image_filter == BLUR:
            result = cv2.blur(frame, (13, 13))
        elif image_filter == FEATURES:
            result = frame
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
            if corners is not None:
                for x, y in np.float32(corners).reshape(-1, 2):
                    cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

        out.write(result)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Apply the filters to the recorded video
apply_filters('video.webm', 'filtered_video.webm')

# Display the filtered video
from IPython.display import Video
Video('filtered_video.webm', embed=True)


<IPython.core.display.Javascript object>

Video saved as video.webm


In [ ]:
import cv2
import numpy as np

# Define the filter types as constants
PREVIEW  = 0  # Preview Mode
BLUR     = 1  # Blurring Filter
FEATURES = 2  # Corner Feature Detector
CANNY    = 3  # Canny Edge Detector

# Define the filters and apply them to the video
def apply_filters(input_path, output_path, filter_type):
    feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the width and height of the frame
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (frame_width, frame_height)

    fourcc = cv2.VideoWriter_fourcc(*'vp80')  # Use 'XVID' or another codec if preferred
    out = cv2.VideoWriter(output_path, fourcc, 20.0, frame_size, isColor=(filter_type != CANNY))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)

        if filter_type == PREVIEW:
            result = frame
        elif filter_type == CANNY:
            result = cv2.Canny(frame, 100, 200, apertureSize=3)  # Adjust apertureSize as needed
            # result = cv2.Canny(frame, 80, 150)
            # result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)  # Convert back to BGR
        elif filter_type == BLUR:
            result = cv2.blur(frame, (13, 13))
        elif filter_type == FEATURES:
            result = frame.copy()
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
            if corners is not None:
                for x, y in np.float32(corners).reshape(-1, 2):
                    cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

        out.write(result)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Choose a filter type: PREVIEW, CANNY, BLUR, FEATURES
filter_type = FEATURES  # Change this to BLUR, FEATURES, or PREVIEW to apply different filters

# Apply the filters to the recorded video
apply_filters('video.webm', 'filtered_video.webm', filter_type)

# Display the filtered video
from IPython.display import Video
Video('filtered_video.webm', embed=True)
